In [2]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import database
from custom_scripts import preprocessing
from custom_scripts import preparation
from custom_scripts import modeling

## Quering the database

In [16]:
example_query =  """
                SELECT * FROM flights
                    LIMIT 10;
                """

example = database.query(example_query)
example.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-05-15,DL,DL,DL,2235,DL,N343NW,2235,11433,DTW,...,1481.0,24.0,0.0,0.0,0.0,136.0,None,NaN,NaN,None


## Fetching predetermined sets of data

In [3]:
all_january = preprocessing.get_all_january_data()
print(all_january.shape)
all_january.head(1)

(1239868, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-01-01,OH,N575NN,5606,13577,11057,2157,2311,74.0,N,1.0,157.0,-3.0


In [4]:
january_2018 = preprocessing.get_2018_january_data()
print(january_2018.shape)
january_2018.head(1)

(601219, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2018-01-01,OH,N720PS,5496,11278,13296,2215,2340,85.0,N,1.0,406.0,1.0


In [6]:
january_2019 = preprocessing.get_2019_january_data()
print(january_2019.shape)
january_2019.head(1)

(638649, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-01-03,MQ,N820AE,3607,15074,11298,1152,1304,72.0,N,1.0,225.0,NaN


In [7]:
first_last_week = preprocessing.get_1st_last_week()
print(first_last_week.shape)
first_last_week.head(1)

(150347, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-12-25,AS,N508AS,800,13830,14893,1155,1856,301.0,N,1.0,2404.0,-10.0


In [8]:
second_last_week = preprocessing.get_2nd_last_week()
print(second_last_week.shape)
second_last_week.head(1)

(154981, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-12-18,AA,N949AN,1584,10423,11298,1240,1347,67.0,N,1.0,190.0,-1.0


In [9]:
third_last_week = preprocessing.get_3rd_last_week()
print(third_last_week.shape)
third_last_week.head(1)

(151975, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-12-11,WN,N7886A,2250,10140,13796,805,945,160.0,N,1.0,889.0,-14.0


In [10]:
fourth_last_week = preprocessing.get_4th_last_week()
print(fourth_last_week.shape)
fourth_last_week.head(1)

(151854, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-12-04,WN,N8556Z,20,11259,12892,1310,1440,210.0,N,1.0,1246.0,5.0


In [11]:
fifth_last_week = preprocessing.get_5th_last_week()
print(fifth_last_week.shape)
fifth_last_week.head(1)

(148053, 13)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,arr_delay
0,2019-11-27,DL,N3752,1282,14747,12892,1704,1945,161.0,N,1.0,954.0,-11.0


## Splitting the data with size and random seed held in script

In [10]:
X = all_january[['origin_airport_id','dest_airport_id']]
y = all_january['arr_delay']
X_train, X_test, y_train, y_test = preparation.get_train_test_split(X, y)

## Using the modeling script to run a test, automatically saving the scores and model to local drive

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
notes = "This is an example"
modeling.run_test(X_train,X_test,y_train,y_test, model, notes)

{'r2_score': -2.1065611590653788e-06, 'mean_squared_error': 2523.296610862557, 'mean_absolute_error': 24.625005634594437, 'explained_variance_score': 6.810394955536481e-06}


LinearRegression()

## Retrieving the records of past modeling tests

In [12]:
modeling.get_records()

,model,notes,training_time,r2_score,mean_squared_error,mean_absolute_error,explained_variance_score
1.pickle,LinearRegression,This is an example,0.043895,-0.000002,2523.296611,24.625006,0.000007


## Retrieving saved model

In [13]:
saved_model = modeling.get_pickle(1)